### Basic code for 2 Hidden Layer Netowork for MNIST sample set 

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np


In [2]:
#Data Locations
train_path = './data/mnist_example/digitstrain.csv'
val_path = './data/mnist_example/digitsvalid.csv'

In [3]:

train_data = pd.read_csv(train_path)
X_train = np.asarray(train_data.values[:,:784])
Y_train = np.asarray(train_data.values[:,784])
print(Y_train.shape)

(2999,)


In [4]:
n_inputs = 784
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10


## Step 1: Making the TF Graph

In [5]:
X = tf.placeholder(tf.float32, shape = [None, n_inputs], name = "X")
y = tf.placeholder(tf.int64, shape = [None], name = "y")

In [6]:
#Method-1 : Create your own funtion
def neuron_layer(X, n_neurons, name, activation = None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs + n_neurons)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev = stddev)
        W = tf.Variable(init, name = "kernel")
        b = tf.Variable(tf.zeros([n_neurons]), name = "bias")
        Z = tf.matmul(X,W) +b
        if activation == None:
            return Z
        else:
            return activation(Z)


with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name = "hidden1", activation = tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name= "hidden2", activation = tf.nn.relu)
    logits = neuron_layer(hidden2, n_outputs, name = "outputs")

In [7]:
#Method -2: Use tf.layers.dense.

with tf.name_scope("dnn2"):
    hidden1 = tf.layers.dense(X, n_hidden1, name = "hidden1", activation =tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name = "hidden2", activation = tf.nn.relu)
    logits = tf.layers.dense(hidden2, n_outputs, name= "logits", activation=tf.nn.elu)

In [8]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y,
                                                             logits = logits)
    loss = tf.reduce_mean(xentropy, name = "loss")

In [9]:
learning_rate = 0.01
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate= learning_rate)
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()


    

## Step 2: Execution of TF

In [10]:
batch_size = 100
val_data = pd.read_csv(val_path)
test = val_data.values 
train_data = train_data.values
np.random.shuffle(train_data)
with tf.Session() as sess:
    init.run()
    
    for epoch in range(100):
        np.random.shuffle(train_data)
        ind = 0
        while ind < train_data.shape[0]:
            X_batch = train_data[ind:ind+batch_size, :784]
            y_batch = train_data[ind:ind+batch_size,784]
            sess.run(training_op, feed_dict = {X : X_batch, y : y_batch})
            ind += batch_size
        acc_val = accuracy.eval(feed_dict = {X : test[:,:784], y : test[:,784]})
        print("Epoch : {!r} , Validation Accuracy : {!r}".format(epoch, acc_val))
        #print ( "" % epoch, acc_val)
    save_path = saver.save(sess, './trained_models/MLP.ckpt')

Epoch : 0 , Validation Accuracy : 0.3103103
Epoch : 1 , Validation Accuracy : 0.46346346
Epoch : 2 , Validation Accuracy : 0.57657659
Epoch : 3 , Validation Accuracy : 0.66266268
Epoch : 4 , Validation Accuracy : 0.71671671
Epoch : 5 , Validation Accuracy : 0.75675678
Epoch : 6 , Validation Accuracy : 0.77277279
Epoch : 7 , Validation Accuracy : 0.7897898
Epoch : 8 , Validation Accuracy : 0.81381381
Epoch : 9 , Validation Accuracy : 0.82882881
Epoch : 10 , Validation Accuracy : 0.82782781
Epoch : 11 , Validation Accuracy : 0.83683681
Epoch : 12 , Validation Accuracy : 0.83583581
Epoch : 13 , Validation Accuracy : 0.84184182
Epoch : 14 , Validation Accuracy : 0.84184182
Epoch : 15 , Validation Accuracy : 0.84684682
Epoch : 16 , Validation Accuracy : 0.85185188
Epoch : 17 , Validation Accuracy : 0.85685688
Epoch : 18 , Validation Accuracy : 0.86286288
Epoch : 19 , Validation Accuracy : 0.86786789
Epoch : 20 , Validation Accuracy : 0.87187189
Epoch : 21 , Validation Accuracy : 0.87087089
